# **Accès à l'API dans le cloud**

<div class="alert alert-info">
Pour la version dans le cloud, pour des raisons de limitations techniques, le document devra être encodé avec USE en local avant d'être transmis. En effet le modèle de sentence embedding préentraîné USE nécessite plus de RAM que celle disponible dans la solution de cloud utilisée. Sans cette limitation technique, il est bien sûr tout à fait possible d'inclure cet encodage dans l'API, que nous pourrions alors directement requêter avec les documents bruts.
<br><br>
Le sentence embedding avec USE est si performant qu'il permet de capter le sens des documents dans plusieurs langues, sans prétraitements préalables. Ce qui fait que même si notre modèle de prédiction a été entraîné sur des documents en Anglais (préalablement encodés avec USE), il fonctionne également avec des documents dans d'autres langues.
</div>

### **Imports**

In [13]:
import tensorflow_hub as hub
import requests

### **Documents sur lesquels faire les prédictions**

In [14]:
document1 = ["how can i convert an array into a dataframe"]

document2 = ["mon tableau ne s'affiche pas dans la bonne couleur sur chrome"]

document3 = ["Je n'arrive pas à compiler mon fichier apk"]

document4 = ["Quand je lance MLPClassifier, j'ai l'erreur ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet. Comment résoudre le problème"]

# Tags réels : html, css
document5 = ["can we add text to a css (div) or should we need to add that text in html just a simple explanation"]

# Tags réels : javascript, dom
document6 = ["""

I want to log, and modify the values assigned at the innerHTML attribute of all dynamically loaded scripts-tags in the page.

To do this, I have come up with the following code:

Object.defineProperty(HTMLScriptElement.prototype, "innerHTML", {
    set: function (value) {
        console.log(value + 1)
    }
});
When loaded into the page the following instructions:

o = document.createElement('script');
o.innerHTML = 1;
will print '2', therefore, I can 'intercept' and modify the assignments successfully.

However, I don't know to save the result into the original innerHTML variable defined in the parent prototype: HTMLElement so that the body of the script will actually contain show 2 when inspected through the DOM.

I have tried:

Object.defineProperty(HTMLScriptElement.prototype, "innerHTML", {
    set: function (value) {
        HTMLElement.prototype.innerHTML = value +1;
    }
});
But this results in an "illegal invocation" error.

While this:

Object.defineProperty(HTMLScriptElement.prototype, "innerHTML", {
    set: function (value) {
        HTMLScriptElement.prototype.innerHTML = value +1;
    }
});
results in: "Maximum call stack size exceeded".

How can I assign the modified value to the innerHTML attribute of the parent prototype?

"""]

documents = [document1,
             document2,
             document3,
             document4,
             document5,
             document6,
]

### **Chargement du modèle de sentense embedding USE**

In [15]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

### **Encodage des documents**

In [16]:
# Nous stockerons les documents encodés dans cette liste
documents_USE = []

for i in documents:
    document_USE = embed(i)
    document = str(document_USE.numpy()[0])
    documents_USE.append(document)

### **Adresse d'accès à l'API**

In [17]:
# Cette adresse devra être mise à jour si l'instance a été stopée un certain temps
URL_server = 'http://4.157.120.225:80'

### **Fonction de requête à l'API**

In [18]:
def predict_tags(document):
    """
    - Cette fonction permet de requêter notre API.
    - Elle prend en entrée les documents (encodés avec USE
      pour la version dans le cloud).
    - Elle fait une requête à l'API et affiche le retour
      fait par l'API : les tags prédits.
    """
    
    url = f"{URL_server}/predict"  # Chemin à utiliser pour requêter l'API
    params = {"document": document}  # Le document sera passé dans le paramètre "document"

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:  # Si la requête a bien abouti
            print(f"Tags prédits : {response.text}")
        # Réponse du serveur, mais pas celle attendue
        else:
            print(f"La requête a échoué avec le code d'état : {response.status_code}")
    except:  # Aucune réponse, connexion impossible
        print("Connexion impossible. L'API est-elle bien lancée ?")

### **Prédictions**

In [20]:
# Pour indiquer le document concerné
compteur = 1

for i in documents_USE:
    print(f"Document n°{compteur}")
    predict_tags(i)
    compteur += 1

Document n°1
Tags prédits : "dataframe, for-loop, pandas, python"
Document n°2
Tags prédits : "css, html, javascript"
Document n°3
Tags prédits : "android, android-studio, gradle, java"
Document n°4
Tags prédits : "optimization, python, tensorflow"
Document n°5
Tags prédits : "css, html"
Document n°6
Tags prédits : "javascript"
